In [ ]:
import guidance
from guidance import models, gen, one_or_more, select, zero_or_more, regex, optional, capture

model_name = "your_model_name"

model = models.LlamaCpp(f"{model_name}.gguf", n_gpu_layers=-1, n_ctx=2048)

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [ ]:
@guidance(stateless=False)
def burgerOrder(lm):
    lm += 'number=' + select(number_values + ['1'], name='numberNames')
    if main_dish_type_values:
        lm += select(
            [", main_dish_type='" + select(main_dish_type_values, name="burgerTypeName") + "'", ""], name='burgerTypeFlag')
    
    if topping_values:
        lm += select([", toppings=[", ""], name='toppingsFlag')
    
        if lm['toppingsFlag'] != "":
            for i in topping_values[:]:
                lm += topping()
                if not topping_values:
                    lm += ']'
                    break
                lm += select([", ", "]"], name="finishedListToppings")
                if lm['finishedListToppings'] == "]":
                    break
    
    return lm + ")"


@guidance(stateless=False)
def topping(lm):
    lm += "Topping(name="
    if topping_values:
        lm += "'" + select(topping_values, name='toppingName') + "'"
    
    if quantity_values:
        lm += select(
            [", qualifier='" + select(quantity_values, name="qualifierName") + "'", ""], name='qualifierFlag')
    
    if not_values:
        lm += select([", negation=True", ""], name='negationFlag')
    
    lm += ")"
    return lm


@guidance(stateless=False)
def drinkOrder(lm):
    lm += 'number=' + select(number_values + ['1'], name='numberNames')
    
    if drink_type_values:
        lm += select(
            [", drink_type='" + select(drink_type_values, name="drinkTypeName") + "'", ""], name='drinkTypeFlag')

    if size_values:
        lm += select(
            [", size='" + select(size_values, name='drinkSizeName') + "'", ""], name='drinkSizeFlag')
    
    return lm + ")"


@guidance(stateless=False)
def sideOrder(lm):
    lm += 'number=' + select(number_values + ['1'], name='numberNames')
    
    if side_type_values:
        lm += select(
            [", side_type='" + select(side_type_values, name='sideTypeName') + "'", ""], name='sideTypeFlag')
    
    if size_values:
        lm += select(
            [", size='" + select(size_values, name='sideSizeName') + "'", ""], name='sideSizeFlag')
        
    return lm + ")"


@guidance(stateless=False)
def validOrderBurger(lm):
    lm += "["
    first = True
    for i in range(7):
        choices = []
        if main_dish_type_values:
            choices.append("BurgerOrder(")
        if drink_type_values:
            choices.append("DrinkOrder(")
        if side_type_values:
            choices.append("SideOrder(")
    
        if choices:
            if not first:
                lm += ", "
            else:
                first = False
    
            lm += select(choices + [']'], name='choice')
            if lm['choice'] == "BurgerOrder(":
                lm += burgerOrder()
            elif lm['choice'] == "SideOrder(":
                lm += sideOrder()
            elif lm['choice'] == "DrinkOrder(":
                lm += drinkOrder()
            else:
                return lm
        else:
            break
    lm += "]"
    return lm


In [ ]:
instruction_generate_burger = """You are a helpful assistant. You have to take as input a customer order and output a list of the corresponding objects. You should use only the following classes in Python:
      class Topping:
            def __init__(self, name: str, qualifier: Optional[str] = None, negation: Optional[bool] = False) -> None:
            
      class BurgerOrder:
            def __init__(self, number: int = 1, size: Optional[str] = None, main_dish_type: Optional[str] = None, toppings: Optional[List[Topping]] = None) -> None
       
      class DrinkOrder:
            def __init__(self, number: int = 1, drink_type: Optional[str] = None, size: Optional[str] = None) -> None :
      
      class SideOrder:
            def __init__(self, number: int = 1, side_type: Optional[str] = None, size: Optional[str] = None) -> None :
      
      The output should be a list of those objects.\n"
      Here's an example:
      'input': 'i would like a vegan burger with lettuce tomatoes and onions and a large order of sweet potato fries',
      'output': '[BurgerOrder(number=1, main_dish_type='vegan_burger', toppings=[Topping(name='lettuce'), Topping(name='tomato'), Topping(name='onion')]), SideOrder(number=1, side_type='french_fries', size='large')]',"""

In [ ]:
import json
import spacy
from spacy.pipeline import EntityRuler
from spacy.language import Language
import os
import re

def parse_line(line):
    parts = line.strip().split('\t')
    if len(parts) != 2:
        return None, None
    phrase, category = parts
    return phrase, category.strip()

def get_all_ngrams(tokens, max_n):
    ngrams = set()
    for n in range(1, max_n + 1):
        for i in range(len(tokens) - n + 1):
            ngram = " ".join([token.text for token in tokens[i:i + n]])
            ngrams.add(ngram)
    return ngrams

def init_pipeline(dataset="burger"):
    nlp = spacy.load("en_core_web_sm")
    
    ner_ruler = nlp.add_pipe("entity_ruler", before="ner", config={"phrase_matcher_attr": "LOWER"})
    
    def read_file_categories(food_type):
        file_path = f"FoodOrderingDataset/data/{food_type}/alias"
        text_files = [f for f in os.listdir(file_path) if f.endswith('.txt')]
        patterns = []
        for file in text_files:
            path_to_file = os.path.join(file_path, file)
            with open(path_to_file, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        phrase, category_info = parse_line(line)
                        if phrase and category_info:
                            schema = {"pattern": phrase, "label": category_info}
                            patterns.append(schema)
        return patterns

    category_patterns = read_file_categories(dataset)
    ner_ruler.add_patterns(category_patterns)
    
    return nlp

@Language.component("disambiguate_size")
def disambiguate_size(doc):
    new_ents = []
    
    max_drink = max((len(phrase.split()) for phrase in DRINK_KEYWORDS), default=1)
    max_side = max((len(phrase.split()) for phrase in SIDE_KEYWORDS), default=1)
    max_n = max(max_drink, max_side)
    
    for ent in doc.ents:
        if "SIZE" in ent.label_:
            start = max(0, ent.start)
            end = min(len(doc), ent.end + 0)
            context_tokens = []
            while end <= len(doc):
                context_tokens = [token for token in doc[start:end]]
                context_ngrams = get_all_ngrams(context_tokens, max_n)
                if context_ngrams & DRINK_KEYWORDS:
                    new_ent = spacy.tokens.Span(doc, ent.start, ent.end, label="DRINK_" + ent.label_)
                    new_ents.append(new_ent)
                    break
                elif context_ngrams & SIDE_KEYWORDS:
                    new_ent = spacy.tokens.Span(doc, ent.start, ent.end, label="SIDE_" + ent.label_)
                    new_ents.append(new_ent)
                    break
                else:
                    if end == len(doc) and start == ent.start:
                        start -= 1
                    else:
                        end += 1
        else:
            new_ents.append(ent)
    doc.ents = tuple(new_ents)
    return doc

def process_NER(input_order):
    found_categories = []
    doc = nlp(input_order)
    for ent in doc.ents:
        found_categories.append((ent.text, ent.label_))
    return found_categories

nlp = init_pipeline()

[('a', 'number(1)'), ('burger', 'MAIN_DISH_TYPE(ham_burger)'), ('tomatoes', 'TOPPING(tomato)'), ('cheese', 'TOPPING(cheddar)'), ('large', 'SIZE(large)'), ('milk', 'DRINK_TYPE(milk)')]


In [ ]:
import json
import re
from collections import defaultdict

file_path = f'FoodOrderingDataset/output/Ablation_CD_Burger_Results_{model_name}.json'

existing_data = []

with open('FoodOrderingDataset/processed_data/burger_dataset_disambiguation.json', 'r') as file:
    data = json.load(file)

burger_str = "['DRINK_TYPE(chocolate_shake)', 'DRINK_TYPE(coca_cola)', 'DRINK_TYPE(coffee)', 'DRINK_TYPE(diet_coke)', 'DRINK_TYPE(dr_pepper)', 'DRINK_TYPE(hot_cocoa)', 'DRINK_TYPE(iced_tea)', 'DRINK_TYPE(milk)', 'DRINK_TYPE(minute_maid)', 'DRINK_TYPE(pink_lemonade)', 'DRINK_TYPE(root_beer)', 'DRINK_TYPE(seven_up)', 'DRINK_TYPE(strawberry_shake)', 'DRINK_TYPE(vanilla_shake)', 'DRINK_TYPE(zero_sugar_lemonade)', 'MAIN_DISH_TYPE(cheese_burger)', 'burger_TYPE(chicken_sandwich)', 'burger_TYPE(double_cheese_burger)', 'burger_TYPE(ham_burger)', 'burger_TYPE(salad)', 'burger_TYPE(vegan_burger)', 'NOT(not)', 'SIDE_TYPE(apple_slices)', 'SIDE_TYPE(baby_carrots)', 'SIDE_TYPE(curly_fries)', 'SIDE_TYPE(french_fries)', 'SIDE_TYPE(garlic_fries)', 'SIDE_TYPE(sweet_potato_fries)', 'SIZE(extra_large)', 'SIZE(large)', 'SIZE(medium)', 'SIZE(small)', 'TOPPING(all_toppings)', 'TOPPING(bacon)', 'TOPPING(blue_cheese)', 'TOPPING(cheddar)', 'TOPPING(jalapenos)', 'TOPPING(ketchup)', 'TOPPING(lettuce)', 'TOPPING(mayonnaise)', 'TOPPING(mustard)', 'TOPPING(onion)', 'TOPPING(pickle)', 'TOPPING(spread)', 'TOPPING(tomato)', 'number(1)', 'number(10)', 'number(11)', 'number(12)', 'number(13)', 'number(14)', 'number(15)', 'number(2)', 'number(3)', 'number(4)', 'number(5)', 'number(6)', 'number(7)', 'number(8)', 'number(9)', 'quantity(extra)', 'quantity(light)']"
all_burger = "\nMenu: " + burger_str.lower()

items_found_str = re.search(r'\nMenu: \[(.*?)\]', all_burger).group(1)
items = re.findall(r'(\w+)\(([^)]+)\)', items_found_str)
items_dict = defaultdict(list)
for item_type, item_value in items:
    items_dict[item_type].append(item_value)

items_dict = dict(items_dict)
keys_to_extract = ['size', 'main_dish_type', 'topping', 'quantity', 'not', 'drink_type', 'side_type', 'number']

for key in keys_to_extract:
    globals()[f"{key}_values"] = items_dict.get(key, [])
    print(key, globals()[f"{key}_values"])

input_list = []
i = 1

for obj in data:
    input_value = obj.get("input", "No input key found")
    output_value = obj.get("output_extract", "No output key found")
    output_generate = obj.get("output_generate", "No output key found")

    input_augmented_file = input_value + all_burger
    input_list.append((input_value, input_augmented_file, output_generate, 'temp', all_burger))

for i in range(len(input_list)):
    print(i)
    if i > 130:
        break
    (initial_input, input_augmented, expected, used_items_value, used_items_str) = input_list[i]
    
    lm = model + f"""\ 
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction_generate_burger}
    ### Input:
    {input_augmented}

    ### Response:
    """

    ans = lm + capture(validOrderBurger(), "answer")
    
    existing_data.append({
        "input": initial_input,
        "input_augmented": input_augmented,
        "output": ans["answer"],
        "expected": expected,
        "output_NER": used_items_str
    })

with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)


size ['extra_large', 'large', 'medium', 'small']
main_dish_type ['cheese_burger']
topping ['all_toppings', 'bacon', 'blue_cheese', 'cheddar', 'jalapenos', 'ketchup', 'lettuce', 'mayonnaise', 'mustard', 'onion', 'pickle', 'spread', 'tomato']
quantity ['extra', 'light']
not ['not']
drink_type ['chocolate_shake', 'coca_cola', 'coffee', 'diet_coke', 'dr_pepper', 'hot_cocoa', 'iced_tea', 'milk', 'minute_maid', 'pink_lemonade', 'root_beer', 'seven_up', 'strawberry_shake', 'vanilla_shake', 'zero_sugar_lemonade']
side_type ['apple_slices', 'baby_carrots', 'curly_fries', 'french_fries', 'garlic_fries', 'sweet_potato_fries']
number ['1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']
0


<>:60: SyntaxWarning: invalid escape sequence '\ '
<>:60: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_214932/1079616513.py:60: SyntaxWarning: invalid escape sequence '\ '
  """


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131


In [ ]:
def split_orders(s):
    s = s.strip()
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1]
    orders = []
    current = ""
    paren_count = 0
    for char in s:
        if char == "(":
            paren_count += 1
        elif char == ")":
            paren_count -= 1
        if char == "," and paren_count == 0:
            orders.append(current.strip())
            current = ""
        else:
            current += char
    if current.strip():
        orders.append(current.strip())
    return sorted(orders)

In [ ]:
import json

def calculate_accuracy_and_save_mismatches(json_file, output_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    total = len(data)
    correct = 0
    mismatches = []

    for item in data:
        print(split_orders(item['output']))
        output = split_orders(item['output'])
        expected = split_orders(item['expected'])

        if output == expected:
            correct += 1
        else:
            mismatches.append(item)

    accuracy = (correct / total) * 100

    with open(output_file, 'w') as outfile:
        json.dump(mismatches, outfile, indent=4)

    return accuracy

json_file = f'FoodOrderingDataset/output/Ablation_CD_Burger_Results_{model_name}.json'
mismatch_file = f'FoodOrderingDataset/output/Ablation_CD_Burger_Mismatches_{model_name}.json'

accuracy = calculate_accuracy_and_save_mismatches(json_file, mismatch_file)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Mismatches have been saved to: {mismatch_file}")

["BurgerOrder(number=1, main_dish_type='cheese_burger', toppings=[Topping(name='lettuce', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), Topping(name='tomato', qualifier='light', negation=True), )", "SideOrder(number=1, side_type='french_fries', size='large')", "SideOrder(number=1, side_type='french_fries', size='large')", "SideOrder(number=1, side_type='french_fries', si